<a href="https://colab.research.google.com/github/14Pranav/Major-Project/blob/main/Sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Major project/archive(1)/Tweets.csv')

In [ ]:
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df.drop(columns=['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],inplace = True)

In [ ]:
df_filtered=df[df['airline_sentiment'] != 'neutral']
df_filtered.head(20)

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
12,positive,@VirginAmerica This is such a great deal! Alre...
13,positive,@VirginAmerica @virginmedia I'm flying your #f...


In [ ]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install contractions

     |████████████████████████████████| 286kB 7.8MB/s 
     |████████████████████████████████| 327kB 14.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85397 sha256=62473ae3ce832287763755ae5ff899d016122dfd4144e6956696264afe6e9354
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [ ]:
import contractions
def con(text):
  expand=contractions.fix(text)#to expand shortened words
  return expand

In [ ]:
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)# replaces pattern with ''
  return text

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [ ]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip()for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [ ]:
df_filtered.text=df_filtered.text.apply(lambda x:x.lower())
df_filtered.text=df_filtered.text.apply(html_tag)
df_filtered.text=df_filtered.text.apply(con)
df_filtered.text=df_filtered.text.apply(remove_sp)
df_filtered.text=df_filtered.text.apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_filtered['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
x = df_filtered.iloc[:,1] #all the statements fro top to bottom from 'text'
y = df_filtered.iloc[:,0]#all the statements fro top to bottom from 'airline_sentiments'

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(8078,)
(3463,)


In [ ]:
import numpy as np
np.unique(y_train,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([6424, 1654]))

In [ ]:
np.unique(y_test,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([2754,  709]))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
sa_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

In [ ]:
sa_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [ ]:
y_pred = sa_model.predict(x_test)
y_pred

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [ ]:
y_test

3512     negative
10159    negative
9752     negative
2876     negative
14311    negative
           ...   
4709     positive
10002    negative
6425     negative
1646     positive
2576     positive
Name: airline_sentiment, Length: 3463, dtype: object

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)*100

90.41293676003465

In [ ]:
confusion_matrix(y_pred,y_test)

array([[2715,  293],
       [  39,  416]])

In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.99      0.90      0.94      3008
    positive       0.59      0.91      0.71       455

    accuracy                           0.90      3463
   macro avg       0.79      0.91      0.83      3463
weighted avg       0.93      0.90      0.91      3463



In [ ]:
sentiment_m = df[df['airline_sentiment']=='negative']
sentiment_m.iloc[6].values

array(['negative',
       '@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'],
      dtype=object)

In [ ]:
sa_model.predict(['@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'])

array(['negative'], dtype=object)

In [ ]:
!pip install pipreqs

In [ ]:
!pipreqs /content

INFO: Successfully saved requirements file in /content/requirements.txt


In [ ]:
cd /content/drive/MyDrive/Major project

/content/drive/MyDrive/Major project


In [ ]:
import joblib
joblib.dump(sa_model,'Tweets')

['Tweets']

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 8.7MB/s 
     |████████████████████████████████| 163kB 44.8MB/s 
     |████████████████████████████████| 81kB 6.3MB/s 
     |████████████████████████████████| 4.2MB 53.5MB/s 
     |████████████████████████████████| 112kB 58.8MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 122kB 43.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.


In [ ]:
%%writefile Sentiments.py
import streamlit as st
import sklearn
import joblib
model = joblib.load('Tweets')
st.title('Tweets Classifier')
ip = st.text_input('Enter your message')
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Overwriting Sentiments.py


In [ ]:
!nohup streamlit run Sentiments.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://277e2587bfeb.ngrok.io'